### Introduction
The amount of malware is growing exponentially. It considered the main security threat. behaviour analysis one of the techniques that used to detect the malware files. This technique depends on study the behaviour of the files in sandbox environment similar to the targeted enviroment of the file. The sandbox provides isolated enviroment that mintores and tracking the activities of the files during running. The collected data then feeded to an algorithm to use it for detection of malware files.   

### Imports

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import tensorflow as tf
import scikitplot as skplt
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import FalseNegatives, TrueNegatives, TruePositives, FalsePositives
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.layers import Embedding, LSTM, Input
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPool1D, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

plt.style.use('ggplot')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/malware-analysis-datasets-api-call-sequences/dynamic_api_call_sequence_per_malware_100_0_306.csv')

In [ ]:
data.head()

The data is 102 columns. The first columns is the hash of the file and it is unique for every file so we can not depend on it as feature in the model.<br/>
``` t_0 to t_99 ``` represent 100 time steps. At each time steps, the API calls reported  from the parent process only and not included the repeated API calls to avoid tracking loops.<br/> The author of the dataset built a list of unique API calls considering
all the samples, and then converted each API call name into a unique integer identifier equal to the index of the API call name in the list. As a result, 307 distinct API calls were identified.

In [ ]:
used_data = data.drop(['hash', 'malware'], axis=1)

In [ ]:
used_data.head()

### Classes distribution

In [ ]:
sns.countplot(data['malware'])

### splitting the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(used_data, data['malware'], test_size=0.25, shuffle=True, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=42)

### Resample the data
We will resample the data to get more balanced data

#### Oversampling

In [ ]:
ros = RandomOverSampler(random_state=0)
X_os, y_os = ros.fit_resample(X_train, y_train)

#### Undersampling

In [ ]:
rus = RandomUnderSampler(random_state=0)
X_us, y_us = rus.fit_resample(X_train, y_train)

### Building the model

In [ ]:
unique_api_calls = 307

In [ ]:
def cnn_lstm_model():
    
    input_ = Input(shape=(100,))
    x = Embedding(input_dim=unique_api_calls, output_dim=8,
                  input_length=X_train.shape[1], name='layer_embedding')(input_)
    x = BatchNormalization()(x)
    x = Conv1D(filters = 32, kernel_size = 9, padding = 'same', activation = 'relu')(x)
    x = MaxPool1D(pool_size = 2)(x)
    x = LSTM(units=512, return_sequences=False, dropout=0.2)(x)
    output = Dense(units=1, activation='sigmoid')(x)
    model = Model(input_, output)
    
    return model

In [ ]:
cnn_lstm_model().summary()

In [ ]:
class Metrics(tf.keras.callbacks.Callback):
    
    def __init__(self, valid_data):
        super(Metrics, self).__init__()
        self.valid_data = valid_data
        
    def on_train_begin(self, logs={}):
        self._data = []
    
    def print_classification_report(self, epoch):
        if epoch%10 == 0:
            X_val, y_val = self.valid_data[0], self.valid_data[1]
            y_predict = self.model.predict(X_val)
            y_predict = np.array([1 if x>0.5 else 0 for x in y_predict])
            print(f'\n epoch {epoch+1} classification report:\n{classification_report(np.array(y_val), y_predict)}\n') 

    
    def on_epoch_end(self, epoch, logs=None):
        self.print_classification_report(epoch)
        return
            

    def get_data(self):
        return self._data




In [ ]:
optimizer = Adam(lr=.0001)
metrics = Metrics(valid_data=(X_val,y_val))

### Training models

#### Imbalnced data model

In [ ]:
imbalaced_model = cnn_lstm_model()
imbalaced_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
history_im = imbalaced_model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=100, batch_size=512, 
                                 callbacks=metrics, verbose=1)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=[12,6])
ax[0].plot(history_im.history["loss"])
ax[0].plot(history_im.history["val_loss"])
ax[0].set_title(" Loss")
ax[0].legend(("Training", "validation"), loc="upper right")
ax[0].set_xlabel("Epochs")
ax[1].plot(history_im.history["accuracy"])
ax[1].plot(history_im.history["val_accuracy"])
ax[1].legend(("Training", "validation"), loc="lower right")
ax[1].set_title("Accuracy")
ax[1].set_xlabel("Epochs")

#### Over sampling balanced model

In [ ]:
os_balnced_model = cnn_lstm_model()
os_balnced_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
history_os_b = os_balnced_model.fit(X_os, y_os, validation_data=(X_val,y_val), epochs=100, batch_size=512, callbacks=metrics)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=[12,6], dpi=120)
ax[0].plot(history_os_b.history["loss"])
ax[0].plot(history_os_b.history["val_loss"])
ax[0].set_title(" Loss")
ax[0].legend(("Training", "validation"), loc="upper right")
ax[0].set_xlabel("Epochs")
ax[1].plot(history_os_b.history["accuracy"])
ax[1].plot(history_os_b.history["val_accuracy"])
ax[1].legend(("Training", "validation"), loc="lower right")
ax[1].set_title("Accuracy")
ax[1].set_xlabel("Epochs")

#### Under sampling balanced model

In [ ]:
us_balnced_model = cnn_lstm_model()
us_balnced_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
history_us_b = us_balnced_model.fit(X_us, y_us, validation_data=(X_val,y_val), epochs=100, batch_size=512, callbacks=metrics)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=[12,6], dpi=120)
ax[0].plot(history_us_b.history["loss"])
ax[0].plot(history_us_b.history["val_loss"])
ax[0].set_title(" Loss")
ax[0].legend(("Training", "validation"), loc="upper right")
ax[0].set_xlabel("Epochs")
ax[1].plot(history_us_b.history["accuracy"])
ax[1].plot(history_us_b.history["val_accuracy"])
ax[1].legend(("Training", "validation"), loc="lower right")
ax[1].set_title("Accuracy")
ax[1].set_xlabel("Epochs")

### Evaluating models

In [ ]:
y_pred_imb = imbalaced_model.predict(X_test)
pred_classes_imb = np.array([0 if i<0.5 else 1 for i in y_pred_imb])

In [ ]:
y_pred_osb = os_balnced_model.predict(X_test)
pred_classes_osb = np.array([0 if i<0.5 else 1 for i in y_pred_osb])

In [ ]:
y_pred_usb = us_balnced_model.predict(X_test)
pred_classes_usb = np.array([0 if i<0.5 else 1 for i in y_pred_usb])

#### Classification Report 

In [ ]:
print(f"classification report for imbalanced data model: \n{classification_report(np.array(y_test), pred_classes_imb)}")
print(f"classification report for over sampled balanced data model: \n{classification_report(np.array(y_test), pred_classes_osb)}")
print(f"classification report for under sampled balanced data model: \n{classification_report(np.array(y_test), pred_classes_usb)}")



#### Confusion Matrix

In [ ]:
fig, ax = plt.subplots(1,3, figsize=[16,8], dpi=440)
skplt.metrics.plot_confusion_matrix(y_test, pred_classes_imb, ax=ax[0])
tickx=ax[0].set_xticklabels(['Benign', 'Malware'])
ticky=ax[0].set_yticklabels(['Benign', 'Malware'])
title=ax[0].set_title('Imbalanced Model')
skplt.metrics.plot_confusion_matrix(y_test, pred_classes_osb, ax=ax[1])
tickx=ax[1].set_xticklabels(['Benign', 'Malware'])
ticky=ax[1].set_yticklabels(['', ''])
labely=ax[1].set_ylabel('')
title=ax[1].set_title('Over sampled balanced Model')

skplt.metrics.plot_confusion_matrix(y_test, pred_classes_usb, ax=ax[2])
tickx=ax[2].set_xticklabels(['Benign', 'Malware'])
ticky=ax[2].set_yticklabels(['', ''])
labely=ax[2].set_ylabel('')
title=ax[2].set_title('under sampled balanced Model')

cb1=fig.axes[0].images[0].colorbar
cb2=fig.axes[1].images[0].colorbar
cb3=fig.axes[2].images[0].colorbar

cb1.remove()
cb2.remove()
cb3.remove()

#### precision_recall Curve

In [ ]:
im_probas = imbalaced_model.predict(X_test, batch_size=64)
im_probas = np.array([[1-im_probas[i][0], im_probas[i][0]] for i in range(len(im_probas))]).reshape(im_probas.shape[0], 2)

os_probas = os_balnced_model.predict(X_test, batch_size=64)
os_probas = np.array([[1-os_probas[i][0], os_probas[i][0]] for i in range(len(os_probas))]).reshape(os_probas.shape[0], 2)

us_probas = us_balnced_model.predict(X_test, batch_size=64)
us_probas = np.array([[1-us_probas[i][0], us_probas[i][0]] for i in range(len(us_probas))]).reshape(us_probas.shape[0], 2)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=[18,8], dpi=440)
skplt.metrics.plot_precision_recall(np.array(y_test), im_probas, ax=ax[0])
title=ax[0].set_title('Imbalanced Model')
skplt.metrics.plot_precision_recall(np.array(y_test), os_probas, ax=ax[1])
title=ax[1].set_title('Over sampled balanced Model')
skplt.metrics.plot_precision_recall(np.array(y_test), us_probas, ax=ax[2])
title=ax[2].set_title('Under sampled balanced Model')
